Salency maps para NLP utilizando AllenNLP
=========================================

<div class="admonition warning">PRECAUCIÓN 😱: El tema presentado en esta sección está clasificado como avanzado. El entendimiento de este contenido es totalmente opcional.</div>

Introducción
------------

AllenNLP es un framework general de aprendizaje profundo para NLP, establecido por el mundialmente famoso Allen Institute for AI Lab. Contiene modelos de referencia de última generación que se ejecutan sobre el `PyTorch`. AllenNLP es una librería que ademas busca implementar abstracciones que permitan el rápido desarrollo de modelos y reutilización de componentes al despegarse de detalles de implementación de cada modelo.

En este ejemplo, veremos como utilizar esta librería para generar salency maps utilizando los gradientes de las prediciones. Esto nos permita interpretar las predicciones de nuestros modelos basados en `transformers`.

### Para ejecutar este notebook

Para ejecutar este notebook, instale las siguientes librerias:

In [2]:
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/NLP/Datasets/mascorpus/tweets_marketing.csv \
    --quiet --no-clobber --directory-prefix ./Datasets/mascorpus/

!wget https://raw.githubusercontent.com/santiagxf/M72109/master/m72109/nlp/explanation.py \
    --quiet --no-clobber --directory-prefix ./m72109/nlp/
    
!wget https://raw.githubusercontent.com/santiagxf/M72109/master/docs/nlp/neural/allennlp_interpret.txt \
    --quiet --no-clobber
%pip install -r allennlp_interpret.txt --quiet

     |████████████████████████████████| 3.1 MB 4.4 MB/s 
     |████████████████████████████████| 831.4 MB 2.6 kB/s 
     |████████████████████████████████| 719 kB 57.2 MB/s 
     |████████████████████████████████| 596 kB 53.4 MB/s 
     |████████████████████████████████| 880 kB 52.3 MB/s 
     |████████████████████████████████| 3.3 MB 39.5 MB/s 
     |████████████████████████████████| 86 kB 4.3 MB/s 
     |████████████████████████████████| 125 kB 73.3 MB/s 
     |████████████████████████████████| 1.8 MB 44.9 MB/s 
     |████████████████████████████████| 592 kB 57.0 MB/s 
     |████████████████████████████████| 248 kB 63.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 1.2 MB 47.7 MB/s 
     |████████████████████████████████| 132 kB 64.7 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |█████████████████

Si ejecuta en Google Colab, adicionalmente deberá cambiar la version de la libraria `google-cloud-storage`:

In [3]:
%pip install -U google-cloud-storage==1.40.0 --quiet

     |████████████████████████████████| 104 kB 2.6 MB/s 
     |████████████████████████████████| 75 kB 5.0 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 1.21.0 requires google-resumable-media!=0.4.0,<0.5.0dev,>=0.3.1, but you have google-resumable-media 1.3.3 which is incompatible.


Descargaremos un modelo previamente entrenando el el problema de clasificación de Tweets:

In [4]:
!wget https://santiagxf.blob.core.windows.net/public/models/tweet_classification_bert.zip --no-clobber --quiet
!unzip -qq tweet_classification_bert.zip

Cargamos el set de datos

In [5]:
import pandas as pd

tweets = pd.read_csv('Datasets/mascorpus/tweets_marketing.csv')

Cargando un modelo entreando con Transformers en AllenNLP
---------------------------------------------------------

`allennlp` es un framework compatible con la libraría `transformers` lo cual resulta atractivo a la hora de utilizar modelos que son entrenados en una para luego llevarlo a la otra. Veamos entonces como podemos hacer para cargar el modelo que tenemos previamente entrenado para la clasificación de tweets utilizando una arquitectura `BERT` dentro de este framework. En particular, nuestro modelo se persistió en el directorio "tweet_classification".

In [6]:
model_name = "tweet_classification_bert"

El detalle de como utilizar AllenNLP esta fuera de este curso, pero utilizaremos el format JSON para cargar modelos de esta libreria. El siguiente codigo carga un modelo exactamente igual al que creamos utilizando la libraria HuggingFace anteriormente.

In [7]:
from allennlp.common import Params
from allennlp.data.dataset_readers import DatasetReader

params = Params({
      "type": "text_classification_json",
      "tokenizer": {
          "type": "pretrained_transformer",
          "model_name": model_name,
      },
      "token_indexers": {
          "tokens": {
              "type": "pretrained_transformer",
              "model_name": model_name,
          }
      }
})

dataset_reader = DatasetReader.from_params(params)

In [15]:
from allennlp.common import Params
from allennlp.models import Model
from transformers import AutoModelForSequenceClassification

params = Params({
    "type": "basic_classifier",
    "vocab": {
        "type": "from_pretrained_transformer",
        "model_name": model_name,
    },
    "text_field_embedder": {
        "type": "basic",
        "token_embedders": {
            "tokens": {
                "type": "pretrained_transformer",
                "model_name": model_name
            }
        }
    },
    "seq2vec_encoder": {
        "type": "bert_pooler",
        "pretrained_model": model_name
    },
    "dropout": 0.1,
    "num_labels": 5,
});

model = Model.from_params(params)
model._classification_layer.weight = AutoModelForSequenceClassification.from_pretrained(model_name).classifier.weight
model._classification_layer.bias = AutoModelForSequenceClassification.from_pretrained(model_name).classifier.bias
_ = model.eval()

In [19]:
from allennlp.predictors import TextClassifierPredictor

predictor = TextClassifierPredictor(model, dataset_reader)

Recordemos que en el conjunto de datos de entrenamiento, las etiquetas se distribuyen como sigue:

In [20]:
labels = [
    'ALIMENTACION',
    'AUTOMOCION',
    'BANCA',
    'BEBIDAS',
    'DEPORTES',
    'RETAIL',
    'TELCO'
]

Interpretando nuestras predicciones
-----------------------------------

Una vez que tenemos nuestro modelo correctamente cargado, veamos como podemos interpretar una predicción computando el salency map a partir de los gradientes.

In [21]:
from allennlp.interpret.saliency_interpreters import SimpleGradient, IntegratedGradient, SmoothGradient

interpreter = SmoothGradient(predictor)

Busquemos un tweet para interpretar:

In [22]:
sample_text_idx = 1522
sample_text = tweets['TEXTO'][sample_text_idx]
sample_label = tweets['SECTOR'][sample_text_idx]

print("Texto:", sample_text, "\Sector:", sample_label)

Texto: @HyundaiPeru con Grupo Primax realiza este verano servicios de Inspección Digital Gratuita a vehículos Hyundai en e… https://t.co/TZ4XFziOd3 \Sector: AUTOMOCION


Calculemos los gradientes para cada token:

In [23]:
import numpy as np

In [24]:
interpretation = interpreter.saliency_interpret_from_json({"sentence": sample_text })
outputs = predictor.predict(sample_text)
grads = np.array(interpretation['instance_1']['grad_input_1'])
probs = np.array(outputs['probs'])

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


In [25]:
outputs.keys()

dict_keys(['logits', 'probs', 'token_ids', 'label', 'tokens'])

Podemos graficar los resultados utilizando un mapa de calor marcando con colores más intensos aquellos tokens que tienen mayor impacto en las predicciones:

In [28]:
from IPython.display import HTML
from eli5.formatters import format_as_html
from m72109.nlp.explanation import get_explanation_from_grads

In [29]:
expl = get_explanation_from_grads(estimator_name="transformer",
                                  estimator_description="NLP transformer explanation",
                                  text=sample_text,
                                  tokens=outputs['tokens'],
                                  grads=grads,
                                  probas=probs,
                                  labels=labels)

In [31]:
HTML(format_as_html(expl))

In [30]:
expl = get_explanation_from_grads(estimator_name="transformer",
                                  estimator_description="NLP transformer explanation",
                                  text=sample_text,
                                  tokens=outputs['tokens'],
                                  grads=grads,
                                  probas=probs,
                                  labels=labels,
                                  force_weights=True)

In [ ]:
HTML(format_as_html(expl, force_weights=True))

Weight?,Feature
+0.011,[CLS]
+0.000,@
+0.000,hyun
+0.029,##da
+0.062,##ipe
+0.029,##ru
+0.000,con
+0.044,grupo
+0.000,prima
+0.000,##x
